# Using ML for paramter estimation

In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sl
from scipy import stats
from sklearn import datasets
from sklearn import linear_model
np.random.seed(1333)

"""Loading the Data"""

dir       = 'data/'
file_name = 'grid1_zheb51fo.xlsx'
UX1       = pd.read_excel(dir+file_name, sheet_name='UX1_Index')
UX2       = pd.read_excel(dir+file_name, sheet_name='UX2_Index')
UX1       = UX1.set_index('Date')
UX2       = UX2.set_index('Date')
UX1.sort_index(inplace=True)
UX2.sort_index(inplace=True)
dataset = np.array(UX1.PX_LAST)
dataset = dataset[4:]              #we drop 4 data for simpler illustratoins

In [2]:
"""Specifying the Input & Output (Labels)"""

n=0     #Looking at n previous days to estimate paramteres
X=np.array([[dataset[j] for j in range(i, i+n+1)] for i in range(len(dataset) - n)])
Y=np.array([dataset[i+n] for i in range(len(dataset) - n)])

In [3]:
"""Splitting Data into Train and Test set"""

m_training= 2000
m_test= 1000

X_training=X[:m_training]
Y_training=Y[:m_training]

X_test=X[m_training:]
Y_test=Y[m_training:]

In [4]:
"""Building the Hypothesis"""

def heston_pde_milstein(V0, r, k, theta, rho, sigma):
    WT  = np.sqrt( 1 ) * np.random.multivariate_normal( np.array( [0,  0] ), np.array( [[1, rho], [rho, 1]] ), size=1)
    V1 = np.abs(V0+ k * (theta - V0) * 1 + sigma * np.sqrt(V0) * WT[:, 1] + .25 * sigma**2 * (WT[:,1]**2 - 1))
    return V1

In [5]:
#Paramteres
theta   = 1
k       = 1
sigma   = 1
rho     = 1
r       = 0    
V0 = 3.5

heston_pde_milstein(V0, r, k, theta, rho, sigma)[0]

1.1794806371367934

In [6]:
X_training= np.ravel(X_training)

Y_pred = np.array([heston_pde_milstein(X_training[i], r, k, theta, rho, sigma)[0] for i in range(len(X_training))])

In [10]:
"""Building the Loss Function"""
#The difference between real label and the predicted one

# l = (heston_pde_milstein(X_training[i], r, k, theta, rho, sigma) - Y_training[i])**2
m=len(X_training)

Ls= (1/m) * np.sum(np.array([(heston_pde_milstein(X_training[i], r, k, theta, rho, sigma) - Y_training[i])**2 
                             for i in range(len(X_training))]))

In [ ]:
"""ERM: Empirical Risk Minimization"""

#we should minimize Ls